In [71]:
# Import necessary packages
import pandas as pd

# Load the data
input_csv_file = input('Enter the name of the csv file : ')
df = pd.read_csv(input_csv_file, encoding="ISO-8859-1")

In [3]:
df.shape
name = 'superstore_final_dataset'

(9800, 18)

In [87]:
def is_not_number(value):
    try:
        float_value = float(value)
        return not isinstance(float_value, (int, float))
    except ValueError:
        return True

In [90]:
# Randomly shuffle the rows in the dataframe 
shuffled_df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Number of lines for json
json_percent = (input('Enter the JSON percentage : '))
json_percent = 30 if is_not_number(json_percent) else json_percent
line_json = int((shuffled_df.shape[0] * json_percent) / 100)

# Number of lines for csv
csv_percent = (input('Enter the CSV percentage : '))
csv_percent = 40 if is_not_number(csv_percent) else csv_percent
line_csv = int((shuffled_df.shape[0] * csv_percent) / 100)

# Number of lines for database
db_percent = (input('Enter the database percentage : '))
db_percent = 30 if is_not_number(db_percent) else db_percent
line_db = int((shuffled_df.shape[0] * db_percent) / 100)

### Json 

In [31]:
# Take 30% to json
json_df = shuffled_df[:line_json]

# Delete the json df from the shuffled
shuffled_df.drop(json_df.index, axis='rows', inplace=True)

# Create json file
json_df.reset_index(drop=True).to_json('json_file.json')

In [32]:
shuffled_df.shape

(6860, 18)

In [33]:
json_df.shape

(2940, 18)

### CSV

In [34]:
# 40% for CSV
csv_df = shuffled_df[:line_csv]

# Delete the csv df to the shuffle
shuffled_df.drop(csv_df.index, axis='rows', inplace=True)

# Create the csv file
csv_df.reset_index(drop=True).to_csv('csv_file.csv', index=False, header=False)

In [35]:
shuffled_df.shape

(2940, 18)

In [36]:
csv_df.shape

(3920, 18)

### Database

In [30]:
# 30% for DB
db_df = shuffled_df[:line_db]

# Delete the csv df to the shuffle
shuffled_df.drop(db_df.index, axis='rows', inplace=True)

2940.0

In [68]:
# Import SQL Alchemy
from sqlalchemy import create_engine, Column, Integer, String, Text, Boolean, Float, Date, text, inspect, UniqueConstraint
from sqlalchemy.orm import sessionmaker, declarative_base

# Construct the connection string
driver = 'ODBC+Driver+18+for+SQL+Server'
server_address = 'localhost'
database_name = 'master' 
username = 'SA' 
password = 'YourPassword123'  

# Connection string for master db
master_connection_string = f'mssql+pyodbc://{username}:{password}@{server_address}/{database_name}?driver={driver}&Encrypt=No'

# Engine for master
master_engine = create_engine(master_connection_string)

In [49]:
# Create class name
def class_name(filename):
    filename = str.replace(filename, '.csv', '')
    filename = str.replace(filename, '_', ' ')
    filename = str.replace(filename, '-', ' ')

    filename = str.title(filename)
    filename = str.replace(filename, ' ', '')
    return filename

In [ ]:
# Transform date type
db_df['Order_Date'] = pd.to_datetime(db_df['Order_Date'])
db_df['Ship_Date'] = pd.to_datetime(db_df['Ship_Date'])

In [39]:
# Create master session
MasterSession = sessionmaker(bind=master_engine)
master_session = MasterSession()

# Retrieve existing databases
get_db_query = 'name FROM sys.databases;'

result = master_session.query(text(get_db_query)).all()

# Get user database
user_db = input('Enter the database name : ')

table_exist = any(row[0] == user_db for row in result)

if not table_exist:
    print('Your table database doesn\'t exist. Do you want to create it ?')
    create_db_answer = input('Enter 1 for yes 0 for no : ')
    while(create_db_answer != '1' and create_db_answer != '0'):
        create_db_answer = input('Enter 1 for yes 0 for no : ')
    if create_db_answer == '1':
        print('Creation of database ',user_db)
        master_session.execute(f'CREATE DATABASE {user_db}')
else:
    master_session.close()
    master_engine.dispose()
    # Connection string for user_db db
    user_db_connection_string = f'mssql+pyodbc://{username}:{password}@{server_address}/{user_db}?driver={driver}&Encrypt=No'
    user_engine = create_engine(user_db_connection_string)

    print('Connection to ',user_db,'database passed successfully')
    print(' ---------- Table creation ---------')

    # Create a declarative base
    Base = declarative_base()

    # Create table
    class_name = class_name(input_csv_file)
    class class_name:
        __tablename__ = class_name

        # Columns
        id = Column(Integer, primary_key=True, autoincrement=True)
        row_id = Column(Integer, nullable=False)
        order_id = Column(String(), nullable=False)
        order_date = Column(Date, nullable=False)
        ship_mode = Column(String(), nullable=False)
        customer_id = Column(String(50), nullable=False)
        customer_name = Column(String(), nullable=False)
        segment = Column(String(50), nullable=False)
        country = Column(String(), nullable=False)
        city = Column(String(), nullable=False)
        state = Column(String(), nullable=False)
        postal_code = Column(Float, nullable=True)
        region = Column(String(), nullable=False)
        product_id = Column(String(50), nullable=False)
        category = Column(String(50), nullable=False)
        sub_category = Column(String(50), nullable=False)
        product_name = Column(String(), nullable=False)
        sales = Column(Float, nullable=False)
Base.metadata.create_all(user_engine)


In [69]:
# Check the created tables
inspector = inspect(user_engine)

table_names = inspector.get_table_names()
table_names

NameError: name 'user_engine' is not defined